# Writing Continuous Applications with Structured Streaming Python APIs in Apache Spark

Tutorial for Spark Streaming


At first glance, building a distributed streaming engine might seem as simple as launching a set of servers and pushing data between them. Unfortunately, distributed stream processing runs into multiple complications that don’t affect simpler computations like batch jobs. Fortunately, PySpark 2.4 and Databricks makes this simple!

**Orignal Author**: Michael John

Modified and Ported to PySpark by Jules S. Damji and Mike for the Tutorial

-sandbox
<div style="line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/12/PySparkStructuredStreaming_FINAL.jpg" alt="Structrured Streaming" width="40%" style=>
</div>

## Setup Data

In [4]:
%run "./setup/setup_data"

### PySpark Documentation: [Click here](https://spark.apache.org/docs/latest/api/python/index.html)
 * [DataFrame](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframe#pyspark.sql.DataFrame)
 * [Spark SQL](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types)
 * [Spark SQL Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions)

# Processing Streaming Sensor Data

Structured Streaming is a powerful capability for building end-to-end continuous applications. At a high-level, it offers the following features:

1. __Output tables are always consistent__ with all the records in a prefix (partition) of the data, we will process and count in order.
1. __Fault tolerance__ is handled holistically by Structured Streaming, including in interactions with output sinks.
1. Ability to handle __late and out-of-order event-time data__. 

<img src="https://demo.cloud.databricks.com/files/mjohns/streaming/continuous-apps-1024x366.png" alt="" width="40%"/>

<sub>reference [Structured Streaming Blog](https://databricks.com/blog/2016/07/28/structured-streaming-in-apache-spark.html)</sub>

## 1-Sources

Consider the input data stream as the “Input Table”. Every data item that is arriving on the stream is like a new row being appended to the Input Table.

<img src="https://demo.cloud.databricks.com/files/mjohns/streaming/cloudtrail-unbounded-tables.png" alt="" width="40%"/>

## 2-Continuous Processing & Queries

The developer then defines a query on this source, or input table, _as if it were a static table_ to compute a final result table that will be written to an output sink. Spark automatically converts this batch-like query to a streaming execution plan. This is called incrementalization: Spark figures out what state needs to be maintained to update the result each time a record arrives. Finally, developers specify triggers to control when to update the results. Each time a trigger fires, Spark checks for new data (new row in the input table), and incrementally updates the result.

<img src="https://demo.cloud.databricks.com/files/mjohns/streaming/cloudtrail-structured-streaming-model.png" alt="" width="40%"/>

## 3-Sinks

The last part of the model is output modes. Each time the result table is updated, the developer wants to write the changes to an external system, such as S3, HDFS, or a database. We usually want to write output incrementally. For this purpose, Structured Streaming provides three output modes:

* __Append__: Only the new rows appended to the result table since the last trigger will be written to the external storage. 
* __Complete__: The entire updated result table will be written to external storage, e.g. for aggregates.
* __Update__: Only the rows that were updated in the result table since the last trigger will be changed in the external storage. 

<img src="https://demo.cloud.databricks.com/files/mjohns/streaming/stream-example1-phone-updated.png" width="40%"/>

## A Continuous Application Example using PySpark Structured Streaming APIs

Setup some file paths our S3 bucket for output, checkpoint, and bad records

In [12]:
output_path = "/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/"
checkpoint_path = "/tmp/pydata/Streaming/continuous_streaming/out/iot-stream-checkpoint"
#
#create checkpoint path
#
dbutils.fs.rm(checkpoint_path,True) #overwrite checkpoint
dbutils.fs.mkdirs(checkpoint_path)
#
#
bad_records_path = "/tmp/pydata/Streaming/continuous_streaming/badRecordsPath/streaming-sensor/"
dbutils.fs.rm(bad_records_path, True) #empty dir
dbutils.fs.mkdirs(bad_records_path)

Out[2]: True

### What does the data from the sensors look like?

In [14]:
sensor_path = "/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/"
sensor_file_name = sensor_path + "streaming-sensor_file-1.json"
dbutils.fs.head(sensor_file_name, 233)

[Truncated to first 233 bytes]
Out[3]: '{"timestamp":"2016-08-03 01:32:20.454","deviceId":44,"deviceType":"SensorTypeD","signalStrength":0.6627327869584749}\n{"timestamp":"2016-08-03 01:32:30.579","deviceId":63,"deviceType":"SensorTypeB","signalStrength":0.6419303214810813}'

### Define schemas for incoming stream and outgoing stream

Good best practice to define a schema and not have Spark infer it, for performance reasons. 
Without a schema, Spark will launch couple of jobs: one to read header, and another to read
a good chuck of the partition to validate the schema, ensuring it matches the data.

Now there are options that you can specifiy so that it fails fast, is tolerant, or subsitute missing
 values or mismatch dataytpes with NaN or null.

In [16]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#original input schema
jsonSchema = (
  StructType()
  .add("timestamp", TimestampType()) #event time at the source
  .add("deviceId", LongType())
  .add("deviceType", StringType())
  .add("signalStrength", DoubleType())
)
# modified schema with added columns since we are 
# doing some ETL (transforming and adding extra columns)
# this transformed data will be stored into parquet files
# from which an SQL table can be created for consumption or
# report generation
parquetSchema = (
  StructType()
  .add("timestamp", TimestampType()) #event time at the source
  .add("deviceId", LongType())
  .add("deviceType", StringType())
  .add("signalStrength", DoubleType())
  .add("INPUT_FILE_NAME", StringType()) #file name from which this data item was read
  .add("PROCESSED_TIME", TimestampType())) #time at the executor while processing

### Read Stream from Object Store Source

In this case we're simulating a Kafka live stream by reading in a file at a time. But this could as well be Apache Kafka topics

__Notice: Intentionally slowing this down for tutorial.__

In [18]:
inputDF = ( spark 
          .readStream 
          .schema(jsonSchema) 
          .option("maxFilesPerTrigger", 1)  #slow it down for tutorial
          .option("badRecordsPath", bad_records_path) #any bad records will go here
          .json(sensor_path) #the source
          .withColumn("INPUT_FILE_NAME", input_file_name()) #maintain file path
          .withColumn("PROCESSED_TIME", current_timestamp()) #add a processing timestamp at the time of processing
          .withWatermark("PROCESSED_TIME", "1 minute") #optional: window for out of order data
         )

### Write Stream to Parquet File Sink

In [20]:
query = (inputDF
         .writeStream
         .format("parquet") #our sink to save it for posterity or batch queries if needed
         .option("path", output_path)
         .option("checkpointLocation", checkpoint_path) # add checkpointing for resiliency
         .outputMode("append")
         .queryName("devices") #optionally a query name over write to issue queries against
         .trigger(processingTime='5 seconds')
         .start() 
        )

In [21]:
%fs ls /tmp/pydata/Streaming/continuous_streaming/out/iot-stream/

path,name,size
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/_spark_metadata/,_spark_metadata/,0
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-00d7d4a1-c46c-4141-8b68-2fbe016c3bd0-c000.snappy.parquet,part-00000-00d7d4a1-c46c-4141-8b68-2fbe016c3bd0-c000.snappy.parquet,4718
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-010d3489-2637-4c38-a737-7feebaf9e10e-c000.snappy.parquet,part-00000-010d3489-2637-4c38-a737-7feebaf9e10e-c000.snappy.parquet,4754
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-02b5aa63-cbb1-4a02-8986-c570cb4b8bb6-c000.snappy.parquet,part-00000-02b5aa63-cbb1-4a02-8986-c570cb4b8bb6-c000.snappy.parquet,4756
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-0300f81e-77c0-4f06-9b01-45bbdf0ba583-c000.snappy.parquet,part-00000-0300f81e-77c0-4f06-9b01-45bbdf0ba583-c000.snappy.parquet,4713
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-03e6acbd-4ee1-4a4b-bab3-28e43d07edec-c000.snappy.parquet,part-00000-03e6acbd-4ee1-4a4b-bab3-28e43d07edec-c000.snappy.parquet,4755
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-0832d227-b6c4-47e6-bead-c8345e5df620-c000.snappy.parquet,part-00000-0832d227-b6c4-47e6-bead-c8345e5df620-c000.snappy.parquet,4730
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-09f2e79e-9bf8-44e8-a77d-5ad96defabe9-c000.snappy.parquet,part-00000-09f2e79e-9bf8-44e8-a77d-5ad96defabe9-c000.snappy.parquet,4750
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-0b49bdee-dea5-4597-bca5-f518cb59ddb0-c000.snappy.parquet,part-00000-0b49bdee-dea5-4597-bca5-f518cb59ddb0-c000.snappy.parquet,4713
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-0ba6f60b-237b-4273-af46-c1a7155c49a4-c000.snappy.parquet,part-00000-0ba6f60b-237b-4273-af46-c1a7155c49a4-c000.snappy.parquet,4762


#### Create a temporary table from the input stream so that you can quickly issue SQL queries against it.

In [23]:
inputDF.createOrReplaceTempView("parquet_sensors")

### Run queries againt the temp table created off the input stream

In [25]:
%sql select * from parquet_sensors where deviceType = 'SensorTypeD' or deviceType = 'SensorTypeA'

timestamp,deviceId,deviceType,signalStrength,INPUT_FILE_NAME,PROCESSED_TIME
2016-08-03T01:32:20.454+0000,44,SensorTypeD,0.6627327869584749,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000
2016-08-03T01:32:11.230+0000,36,SensorTypeD,0.8869519587040092,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000
2016-08-03T01:32:29.490+0000,6,SensorTypeA,0.621775542316748,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000
2016-08-03T01:32:26.601+0000,31,SensorTypeA,0.7217142790760191,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000
2016-08-03T01:32:14.999+0000,2,SensorTypeD,0.9463019453554669,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000
2016-08-03T01:32:19.582+0000,31,SensorTypeA,0.4177194090879053,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000
2016-08-03T01:32:14.446+0000,61,SensorTypeA,0.8093833461353955,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000
2016-08-03T01:32:12.688+0000,93,SensorTypeA,0.6009808303441028,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000
2016-08-03T01:32:13.518+0000,45,SensorTypeD,0.705299532046275,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000
2016-08-03T01:32:15.719+0000,68,SensorTypeD,0.39902827917907346,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2020-02-11T02:37:16.021+0000


(Click Run All Above Here)

(Then Individually run below)

### Run queries and additional processing against Parquet file stored from the input stream

In [28]:
spark.conf.set("spark.sql.shuffle.partitions", "1") #keep the size of shuffles small for better query performance
devices = (spark.readStream
           .schema(parquetSchema)
           .format("parquet")
           .option("maxFilesPerTrigger", 1) #slow it down to demo
           .load(output_path)
           .withWatermark("PROCESSED_TIME", "1 minute") #window for out of order data
          )

 # generate temp table for more complex aggregation queries
devices.createOrReplaceTempView("sensors")

#### What files are being processed?

In [30]:
display(
  devices.
  select("INPUT_FILE_NAME", "PROCESSED_TIME")
  .groupBy("INPUT_FILE_NAME", "PROCESSED_TIME")
  .count()
  .orderBy("PROCESSED_TIME", ascending=False)
)

#### How much data is coming through?

In [32]:
%sql select count(*) from sensors

#### What does the min, max, and average strength look like for each sensor type?

Use Spark SQL functions min(), max(), and avg()

Note: I can use SQL within my Python notebook via `%sql` magic command

In [34]:
%sql 

select count(*), deviceType, min(signalStrength), max(signalStrength), avg(signalStrength) 
  from sensors 
    group by deviceType 
    order by deviceType asc

#### Let's create a stream to aggregate signal counts by device and timestamp over 5 second windows.

Note: this is tumbling window, not a sliding window; its size is 5 seconds

For example, discounting day, hours, and min, a tumbling window of size 5 looks as follows:

*[(00:00 - 00:05), (00:05: 00:10), (00:10: 00:15)]*

An event could fall into any of these tumbling windows.

In [36]:
(devices
 .groupBy(
   window("timestamp", "5 seconds"),
   "deviceId"
 )
 .count()
 .createOrReplaceTempView("sensor_counts")) #create a temporary view atop DataFrame

#### Which devices are experiencing signal loss over those 5 second windows?

In [38]:
%sql select * from sensor_counts where count < 5 order by window.start desc

### Send Alerts for potentially down sensors that have not sent signals

Let's create a DataFrame from our temporary table `sensor_counts`

In [40]:
lost_sensor_signals = (spark.table("sensor_counts")
         .filter(col("count") < 5)
         .select("window.start", "window.end", "deviceId", "count")
         )

#display our DataFrame
display(lost_sensor_signals)

##### Using `foreach` mechanism to write to workers' logs.

This could be used for monitoring purposes. Either another job scanning the logs for ALERTS
and publishing them onto a topic on Kafka or posting them on Ganglia. A good excerice to try
if you have a Kafka cluster aviable or Ganglia service available via a REST API.

In [42]:
def processRow(row):
  # for now write them to log files, but this logic can easily be extended to publishing alerts
  # to a topic on Kafka or monitoring/paging service such as Ganglia or PagerDuty.
  print("ALERT from Sensors: Between {} and {}, device {} reported only {} times".format(row.start, row.end, row.deviceId, row[3]))
  
(lost_sensor_signals
 .writeStream
 .outputMode("complete") #could be our Kafka topic "alerts" for monitoring
 .foreach(processRow)
 .start()
)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2024864360423174> in <module> 
 7 . writeStream
 8 . outputMode ( "complete" ) #could be our Kafka topic "alerts" for monitoring 
 ----> 9 . foreach ( processRow ) 
 10 . start ( ) 
 11 )

 /databricks/spark/python/pyspark/sql/streaming.py in start (self, path, format, outputMode, partitionBy, queryName, **options) 
 1120 self . queryName ( queryName ) 
 1121 if path is None : 
 -> 1122 return self . _sq ( self . _jwrite . start ( ) ) 
 1123 else : 
 1124 return self . _sq ( self . _jwrite . start ( path ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o652.start.
: java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 10001); nested exception is: 
	com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 10001)
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:83)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at sun.nio.cs.StreamEncoder.implClose(StreamEncoder.java:320)
	at sun.nio.cs.StreamEncoder.close(StreamEncoder.java:149)
	at java.io.OutputStreamWriter.close(OutputStreamWriter.java:233)
	at com.fasterxml.jackson.core.json.WriterBasedJsonGenerator.close(WriterBasedJsonGenerator.java:883)
	at com.fasterxml.jackson.databind.ObjectMapper._configAndWriteValue(ObjectMapper.java:3561)
	at com.fasterxml.jackson.databind.ObjectMapper.writeValue(ObjectMapper.java:2909)
	at org.json4s.jackson.Serialization$.write(Serialization.scala:27)
	at org.apache.spark.sql.execution.streaming.StreamMetadata$.write(StreamMetadata.scala:80)
	at org.apache.spark.sql.execution.streaming.StreamExecution$$anonfun$2.apply(StreamExecution.scala:128)
	at org.apache.spark.sql.execut

_Here is an example of the results being written out._

<img src="https://databricks.com/wp-content/uploads/2018/12/Screen-Shot-2018-12-18-at-5.53.39-PM.png" alt="" width="50%"/>

##### CHALLENGE-1: 

Try using`foreachBatch` to write each micro-batch using standard DataFrame API.

__See [docs](https://docs.databricks.com/spark/latest/structured-streaming/foreach.html).__

In [45]:
def foreach_batch_function(df, epoch_id):
    # Transform and write batchDF
    for row in df.rdd.collect():
      print("ALERT from Sensors: Between {} and {}, device {} reported only {} times".format(row.start, row.end, row.deviceId, row[3]))
  
(lost_sensor_signals
 .writeStream
 .outputMode("complete") #could be our Kafka topic "alerts" for monitoring
 .foreachBatch(foreach_batch_function)
 .start()
)

##### CHALLENGE-2: 

Try adding additional monitoring conditions on devices. For example, add another streaming query that
filters monitoring conditions (signal strength < 0.2) from any of the above streams or tables: sensor_counts, devices, or inputStream, or parquet files.

__See [docs](https://docs.databricks.com/spark/latest/structured-streaming/foreach.html).__

In [47]:
low_signal_strength = (spark.table("parquet_sensors")
         .filter(col('signalStrength') < 0.2)
         .select("timestamp", "deviceId", "deviceType", "signalStrength")
         )

#display our DataFrame
display(low_signal_strength)

In [48]:
def processRow1(row):
  # for now write them to log files, but this logic can easily be extended to publishing alerts
  # to a topic on Kafka or monitoring/paging service such as Ganglia or PagerDuty.
  print("ALERT from Sensors: at {} device {} reported a low signal strength of {}".format(row.timestamp, row.deviceId, row.signalStrength))
  
(low_signal_strength
 .writeStream
 #.outputMode("complete") #could be our Kafka topic "alerts" for monitoring
 .foreach(processRow1)
 .start()
)

## Cleanup Data

In [50]:
%run "./setup/cleanup_data"